In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import namedtuple, deque 
from random import random, sample, randint

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm

from tic_env import TictactoeEnv, OptimalPlayer

In [3]:
def valid_move(grid, move):
    return grid[(move//3,move%3)] == 0

def act(model, grid, player, epsilon):
    return act_random(grid) if random()<epsilon else act_optimal(model,grid, player)

def act_random(grid):
    actions = [i for i in range(9) if valid_move(grid,i)]
    return actions[randint(0,len(actions)-1)]

def act_optimal(model, grid, player):
     with torch.no_grad():
            actions_reward = model(grid_to_state(grid, player))
            return torch.argmax(actions_reward).item()

def grid_to_state(grid, player):
    return torch.tensor(np.stack([grid>0,grid<0] if player=='X' else [grid<0,grid>0])).float().view(1,-1)

        
def push(memory, last_state, last_move, grid, reward, player):
    if last_move is not None and last_state is not None:
        memory.push(grid_to_state(last_state, player), last_move, None if grid is None else grid_to_state(grid,player), reward)
    


def network():
    return nn.Sequential(nn.Linear(9*2,128),
                        nn.ReLU(inplace=True),
                        nn.Linear(128,128),
                        nn.ReLU(inplace=True),
                        nn.Linear(128,9))

#######from pytroch tutorial
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory:

    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [5]:
env = TictactoeEnv()
grid, _, _ = env.step(0, print_grid=False)
grid_to_state(grid, 'X')

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [12]:
def compute_M(model, epsilon, n_games = 500):
    Turns = np.array(['X','O'])
    env = TictactoeEnv()


    deep_q_turn = Turns[0]
    win_minus_loss = 0
    
    player_opt = OptimalPlayer(epsilon=epsilon, player=Turns[1])
    
    for i in range(n_games):
        grid, _, __ = env.observe()
        
        #QPlayer plays first for the first 250 games and we switch to the optimal player after n_games/2 games
        if i == n_games//2 :
            player_opt.player = Turns[0]
            deep_q_turn = Turns[1]
        
        for j in range(9):
            if env.current_player == player_opt.player:
                move = player_opt.act(grid)
                move = move[0]*3+move[1]
            else:
                with torch.no_grad():
                    move = act_optimal(model, grid, deep_q_turn)
            
           
            if not valid_move(grid,move):
                win_minus_loss -= 1
                env.reset()
                break

            grid, end, winner = env.step(move, print_grid=False)
        
           

            if end:
                win_minus_loss += env.reward(player=deep_q_turn)
                        
                env.reset()
                break
    
   
    return (win_minus_loss)/n_games


def epsilon(n, n_star, epsilon_min = 0.1, epsilon_max = 0.8):
    return np.maximum(epsilon_min, epsilon_max*(1-n/n_star))

def train_on_a_game(optimal, model, epsilon, model_turn, optimize_model, memory):
    env = TictactoeEnv()
    grid, _, __ = env.observe()
    
    sum_loss= 0
    nb_updates = 0
    
    last_move = None
    last_state = None
    
    for j in range(9):
            
            if env.current_player == model_turn:
                move =  act(model, grid, model_turn, epsilon)
               
                push(memory, last_state, last_move, grid, 0, model_turn)
                last_move = move
                last_state = grid
                    
            else:
                move = optimal.act(grid)
                move = move[0]*3+move[1]
                
          
            sum_loss += optimize_model()
            nb_updates += 1
            
            
            
            if not valid_move(grid,move):
                push(memory, last_state, last_move, None, -1, model_turn)
                return sum_loss/max(1,nb_updates), -1
            
                
            grid, end, winner = env.step(move, print_grid=False)
            
 
            
            if end:
                reward = env.reward(player=model_turn)
                push(memory, last_state, last_move, None, reward, model_turn)
                return sum_loss/max(1,nb_updates), reward

def optimize_model(policy_net, target_net, memory , optim, criterion, batch_size, gamma=0.99):
    ##### inspired from :: https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html
    if len(memory)<batch_size:
        return 0
    
    
    transitions = memory.sample(batch_size)
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
  
   
    
    state_batch = torch.cat(batch.state)
    action_batch = torch.tensor(batch.action).unsqueeze(1)
    reward_batch = torch.tensor(batch.reward).unsqueeze(1)

 

    state_action_values = policy_net(state_batch).gather(1, action_batch)
    

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    with torch.no_grad():
        next_state_values = torch.zeros(batch_size)
        
        non_none_states = [s for s in batch.next_state if s is not None]
        if len(non_none_states)>0:
            non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), dtype=torch.bool) 
            non_final_next_states = torch.cat(non_none_states) 
            next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]
            
        next_state_values = next_state_values.view((-1,1))
      
    
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * gamma) + reward_batch


    # Compute Huber loss

    loss = criterion(state_action_values, expected_state_action_values)
    
    # Optimize the model
    optim.zero_grad()
    loss.backward()
   
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optim.step()
   
   
    return loss.item()

In [15]:
def train_deep_q_against_self(epsilon, buffer_size=10000, 
                              batch_size=64, nb_games = 20000, 
                              update_length = 500, 
                              average_length = 250.0):
    
    Turns = np.array(['X','O'])
    
    policy_net = network()
    target_net = network()
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()
    
   
    
    memory = ReplayMemory(buffer_size)
  
    
    optimizer = optim.Adam(policy_net.parameters(), lr=8e-5)
    criterion = nn.HuberLoss()
    
    optimize_m = lambda  : optimize_model(policy_net, target_net, memory,  
                                            optimizer, criterion, batch_size = batch_size, gamma=0.99)
    
    
    M_opts =  []
    M_rands = []
  
    for i in tqdm(range(nb_games)):
        
        
        
        train_on_a_game_against_self(policy_net, epsilon, optimize_m, memory)
        
        
        if np.mod(i+1,update_length) == 0:
            target_net.load_state_dict(policy_net.state_dict())
        
        if np.mod(i+1,average_length) == 0:
            M_opts.append(compute_M(policy_net,0))
            M_rands.append(compute_M(policy_net,1))
           
    return M_opts, M_rands


def train_on_a_game_against_self(model, epsilon,  optimize_model, memory):
    env = TictactoeEnv()
    grid, _, __ = env.observe()
    
    
    last_move_1 = None
    last_state_1 = None
    
    last_move_2 = None
    last_state_2 = None
    
    for j in range(9):
            
            move =  act(model, grid, env.current_player, epsilon)
            
            if env.current_player == 'X':
               
                push(memory, last_state_1, last_move_1, grid, 0, 'X')
                last_move_1 = move
                last_state_1 = grid
                
                    
            else:
              
                push(memory, last_state_2, last_move_2, grid, 0, 'O')
                last_move_2 = move
                last_state_2 = grid
            
            optimize_model()
                
              
            if not valid_move(grid,move) and env.current_player == 'X':
                    push(memory, last_state_1, last_move_1, None, -1, env.current_player)
                    return
                
            if not valid_move(grid,move) and env.current_player == 'O':
                    push(memory, last_state_2, last_move_2, None, -1, env.current_player)
                    return
                
            grid, end, winner = env.step(move, print_grid=False)
            
 
            
            if end:
                reward_1 = env.reward(player='X')
                reward_2 = env.reward(player='O')
                push(memory, last_state_1, last_move_1, None, reward_1, 'X')
                push(memory, last_state_2, last_move_2, None, reward_2, 'O')
                return

In [ ]:
M_opt, M_rand = train_deep_q_against_self(0.1, nb_games=5000)

 46%|█████████████████▉                     | 2307/5000 [01:01<00:58, 45.94it/s]

In [ ]:
plt.plot(M_opt)

In [ ]:
plt.plot(M_rand)